# LLM Prompting

Concepts:
- Financial News Sentiment
- Llama-3
- Ollama
- Kaggle
- Prompting


In [1]:
import pandas as pd
from pandas import DataFrame, Series
import ollama
from secret import paths

## Llama-3 LLM

https://ai.meta.com/blog/meta-llama-3/
https://ollama.com/library/llama3

## Ollama server

Ollama is a tool that helps us run large language models on our local machine and makes experimentation more accessible.  It provides a simple API for creating, running, and managing models, as well as a library of pre-built models.

https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807

https://github.com/ollama/ollama

1. Install Ollama (https://ollama.com/)
   - `curl https://ollama.ai/install.sh | sh`

2. Pull a model
   - `ollama pull llama3:instruct`

   In Linux, the pulled models will be stored at /usr/share/ollama/.ollama/models

3. Serve an LLM

   - `ollama serve` - may not use GPU?!

   - `ollama run llama3:instruct` - use GPU

4. Linux service
```
# sudo systemctl status ollama # service status
# sudo systemctl disable ollama # disable so it does not start up again upon reboot
# sudo systemctl stop ollama # stop service
# sudo systemctl restart ollama # restart service
# sudo rm /etc/systemd/system/ollama.service # delete service file
# sudo rm $(which ollama) # remove ollama binary
```

5. Endpoint
   - `curl http://localhost:11434/api/generate -d '{"model": "llama3:instruct", "prompt":"Why is the sky blue?"}'`



In [2]:
for _ in range(3):
    output = ollama.generate(model="llama3:instruct",
                        prompt="Are you Llama-2 or Llama-3?")
    print(output['response'])

I'm just an AI, not a llama at all! I don't have a version number like LLaMA-2 or LLaMA-3. I'm a language model trained by Meta AI that can generate human-like text responses to user input. Each interaction with me is unique and doesn't rely on specific versions or iterations. So, feel free to chat with me anytime!
I am LLaMA-1. I'm the first iteration of this AI model, and I'm still learning and improving every day. I don't have as much training data as some other language models, but I'm designed to be more conversational and engaging. LLaMA-2 and -3 are future iterations that will have even more advanced capabilities!
I am LLaMA, the third generation of the LLaMA AI models. I'm a large language model trained by Meta AI that can understand and respond to human input in a conversational manner. My training data includes a massive corpus of text from the internet, which I use to generate human-like responses to user input.


## Kaggle platform

A subsidiary of Google, it is an online community for data scientists and machine learning engineers. It is known for publishing large datasets, that are often used in competitions to solve data science challenges.

### A financial news sentiment dataset

https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

This dataset contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in: Malo, P., Sinha, A., Takala, P., Korhonen, P. and Wallenius, J. (2014): “Good debt or bad debt: Detecting semantic orientations in economic texts.” Journal of the American Society for Information Science and Technology.


In [3]:
news = pd.read_csv(paths['data'] / 'all-data.csv',
                   names=["sentiment", "text"],
                   encoding="utf-8",
                   encoding_errors="replace")
news

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [4]:
positive = list(news.index[news['sentiment'].eq('positive')])
neutral = list(news.index[news['sentiment'].eq('neutral')])
negative = list(news.index[news['sentiment'].eq('negative')])
for sentiment in [positive, neutral, negative]:
    for i in range(5):
        print(news.iloc[sentiment[i]]['text'])
        print(f"   => true label={news.iloc[sentiment[i]]['sentiment']}")

With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
   => true label=positive
According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
   => true label=positive
FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
   => true label=positive
For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .
   => true label=positive
In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to 

## Prompting techniques

https://llama.meta.com/docs/how-to-guides/prompting/


### Zero-shot prompt


In [40]:
def generate_prompt(text):
    return f"""
Text: {text}
Sentiment:""".strip()

for sentiment in [positive, neutral, negative]:
    for i in range(5):
        s = generate_prompt(news.iloc[sentiment[i]]['text'])
        output = ollama.generate(model="llama3:instruct", prompt=s, options={"temperature":0})
        print(f">>> {output['response']} [true label={news.iloc[sentiment[i]]['sentiment']}]")
        print()

>>> Positive [true label=positive]

>>> Positive sentiment. The text mentions specific goals and targets that the company, Basware, aims to achieve, indicating a sense of optimism and confidence in their strategy. [true label=positive]

>>> Positive. The text suggests that Aspocomp is actively working towards its growth strategy, which implies a sense of enthusiasm and optimism about the company's future prospects. The use of words like "aggressively pursuing" and "technologically more demanding" also convey a sense of confidence and ambition. Overall, the sentiment is positive and forward-looking. [true label=positive]

>>> Positive. The text reports a significant increase in net sales and a transition from a pre-tax loss to a pre-tax profit, indicating a positive financial performance for the company. [true label=positive]

>>> Positive sentiment.

The text mentions an increase in net sales (5.2%) and operating profit (34.9%), which suggests a positive trend for the company's financi

Write prompt instructions clearly, and specify output requirements, such as json format

In [41]:
def generate_prompt(text):
    return f"""
Classify the sentiment of the following text as "positive" or "neutral" or "negative".
Provide your output in json format. Do not provide any other answer.

Text: {text}
Sentiment:""".strip()

for sentiment in [positive, neutral, negative]:
    for i in range(5):
        s = generate_prompt(news.iloc[sentiment[i]]['text'])
        output = ollama.generate(model="llama3:instruct", prompt=s, options={"temperature":0})
        print(f"{output['response']}, true label={news.iloc[sentiment[i]]['sentiment']}")



{"sentiment": "positive"}, true label=positive
{"sentiment": "neutral"}, true label=positive
{"sentiment": "positive"}, true label=positive
{"sentiment": "positive"}, true label=positive
{"sentiment": "neutral"}, true label=positive
{"sentiment": "neutral"}, true label=neutral
{"sentiment": "neutral"}, true label=neutral
{"sentiment": "neutral"}, true label=neutral
{"sentiment": "neutral"}, true label=neutral
{"sentiment": "positive"}, true label=neutral
{"sentiment": "negative"}, true label=negative
{"sentiment": "negative"}, true label=negative
{"sentiment": "negative"}, true label=negative
{"sentiment": "negative"}, true label=negative
{"sentiment": "negative"}, true label=negative


### Few-shot prompt

Adding specific examples of your desired output generally results in a more accurate, consistent output.


In [50]:
N = 3
examples = positive[-N:] + neutral[-N:] + negative[-N:]
examples = list(news.iloc[examples].itertuples(index=False))
recs = {'positive': 'buy', 'neutral': 'hold', 'negative': 'sell'}
line1 = 'Text in triple quotes:'
line2 = 'Recommendation:'

def generate_prompt(text, examples):
    shots = "\n\n".join([f"{line1} '''{t}'''\n{line2} {recs[s]}"
                         for s,t in examples])
    return f"""
Here are {len(examples)} examples of making a recommendation based on the
sentiment of the given text delimited with triple quotes.

{shots}

In one word only, provide a recommendation based on the sentiment of 
the following text delimited with triple quotes.  
{line1} '''{text}'''
{line2}""".strip()

In [51]:
for sentiment in [positive, neutral, negative]:
    for i in range(5):
        s = generate_prompt(news.iloc[sentiment[i]]['text'], examples)
        output = ollama.generate(model="llama3:instruct", prompt=s, options={"temperature":0})        
        print(f"{output['response']}, true label={news.iloc[sentiment[i]]['sentiment']}")

Buy, true label=positive
Buy, true label=positive
Buy, true label=positive
buy, true label=positive
Buy, true label=positive
Hold, true label=neutral
Buy, true label=neutral
Buy, true label=neutral
Hold, true label=neutral
Buy, true label=neutral
Sell, true label=negative
Sell, true label=negative
Sell, true label=negative
Sell, true label=negative
Sell, true label=negative


Display few-shot prompt

In [52]:
print(s)

Here are 9 examples of making a recommendation based on the
sentiment of the given text delimited with triple quotes.

Text in triple quotes: '''Danske Bank A-S DANSKE DC jumped 3.7 percent to 133.4 kroner , rebounding from yesterday s 3.5 percent slide .'''
Recommendation: buy

Text in triple quotes: '''Our superior customer centricity and expertise in digital services set us apart from our competitors .'''
Recommendation: buy

Text in triple quotes: '''The 2015 target for net sales has been set at EUR 1bn and the target for return on investment at over 20 % .'''
Recommendation: buy

Text in triple quotes: '''It holds 38 percent of Outokumpu 's shares and voting rights , but in 2001 lawmakers gave it permission to reduce the stake to 10 percent .'''
Recommendation: hold

Text in triple quotes: '''Mobile communication and wireless broadband provider Nokia Inc NYSE : NOK today set new financial targets and forecasts for Nokia and the mobile device industry and also for Nokia Siemens Net

### Chain of thought

Provide a series of prompts or
questions to help guide its thinking and generate a more coherent
and relevant response. 


In [45]:
def generate_prompt(text):
    return f"""
You are a financial market analyst who makes a recommendation based on the 
sentiment of the text given in triple quotes. Begin with:
1. Write a summary of the text in about 10 words.
2. Explain its impact on stock price.
3. Provide your assumptions.
4. Describe what might go wrong.
5. Finally, give a recommendation to "buy" or "hold" or "sell".
Make your recommendation funny.

Text: '''{text}'''
Recommendation:""".strip()

In [46]:
for sentiment in [positive, neutral, negative]:
    for i in range(1):
        s = generate_prompt(news.iloc[sentiment[i]]['text'])
        print('==================================')
        print(news.iloc[sentiment[i]]['text'])
        print('----------------------------------')
        output = ollama.generate(model="llama3:instruct", prompt=s, options={"temperature":0})
        print(f"{output['response']}")
        print()
        print()

With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
----------------------------------
Here's my analysis:

**Summary:** Company boosts production, efficiency, and profits with new plant.

**Impact on stock price:** This news is a big positive for the company's stock. Investors will likely respond favorably to increased capacity, reduced waste, and higher profit margins. I expect the stock price to rise 5-7% in the short term (next quarter).

**Assumptions:**

1. The new plant will be operational within the next 6 months.
2. Demand for the company's products will indeed increase as expected.
3. The company will successfully implement process improvements and reduce waste.

**What might go wrong:**

1. Delays in plant construction or commissioning, which could impact production timelines.
2. Unforeseen issues with raw material sourcin